# 趨勢策略入門

### 關鍵函數：
 - 1 交易函數（接收一個交易字典，包含交易價格，交易量，幣種，交易方向等）
 - 2 持倉計算函數（根據倉位情況和風險設定，計算限制交易量以及交易價格）
 - 3 趨勢條件計算函數（根據品種價格情況，判斷是否執行交易，交易方向，交易量，以及交易價格）

In [1]:
class mid_class():
    def __init__(self, this_exchange):
        '''
        初始化數據填充交易所的信息，首次獲取價格，首次獲取account信息
        設定好密鑰……
        
        Args:
            this_exchange: FMZ的交易所結構
        
        '''
        self.init_timestamp = time.time()
        self.exchange = this_exchange
        self.name = self.exchange.GetName()
        self.jyd = self.exchange.GetCurrency()        
    
    def get_account(self):
        '''
        獲取賬戶信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.Balance = '---'
        self.Amount = '---'
        self.FrozenBalance = '---'
        self.FrozenStocks = '---'
        
        try:
            self.account = self.exchange.GetAccount()

            self.Balance =  self.account['Balance']
            self.Amount = self.account['Stocks']
            self.FrozenBalance =  self.account['FrozenBalance']
            self.FrozenStocks = self.account['FrozenStocks']
            return True
        except:
            return False
    
    def get_ticker(self):
        '''
        獲取市價信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.high = '---'
        self.low = '---'
        self.Sell =  '---'
        self.Buy =  '---'
        self.last =  '---'
        self.Volume = '---'
        
        try:
            self.ticker = self.exchange.GetTicker()
        
            self.high = self.ticker['High']
            self.low = self.ticker['Low']
            self.Sell =  self.ticker['Sell']
            self.Buy =  self.ticker['Buy']
            self.last =  self.ticker['Last']
            self.Volume = self.ticker['Volume']
            return True
        except:
            return False
        
        
    def get_depth(self):
        '''
        獲取深度信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.Ask = '---'
        self.Bids = '---'
        
        try:
            self.Depth = self.exchange.GetDepth()
            self.Ask = self.Depth['Asks']
            self.Bids = self.Depth ['Bids']
            return True
        except:
            return False
        
        
    
    def get_ohlc_data(self, period = PERIOD_M5):
        '''
        獲取K線信息
        
        Args:
            period: K線週期，PERIOD_M1 指1分鐘, PERIOD_M5 指5分鐘, PERIOD_M15 指15分鐘,
            PERIOD_M30 指30分鐘, PERIOD_H1 指1小時, PERIOD_D1 指一天。
        '''
        self.ohlc_data = exchange.GetRecords(period)
        
        
    
    def create_order(self, order_type, price, amount):
        '''
        post一個掛單信息
        
        Args:
            order_type：掛單類型，'buy'指掛買單，'sell'指掛賣單
            price：掛單價格
            amount:掛單數量
            
        Returns:
            掛單Id號，可用以取消掛單
        '''
        if order_type == 'buy':
            try:
                order_id = self.exchange.Buy( price, amount)
            except:
                return False
            
        elif order_type == 'sell':
            try:
                order_id = self.exchange.Sell( price, amount)
            except:
                return False
        
        return order_id
    
    def get_orders(self):
        self.undo_ordes = self.exchange.GetOrders()
        return self.undo_ordes
    
    def cancel_order(self, order_id):
        '''
        取消一個掛單信息
        
        Args:
            order_id：希望取消的掛單ID號
            
        Returns:
            取消掛單成功返回True，取消掛單失敗返回False
        '''
        return self.exchange.CancelOrder(order_id)
        
    def refreash_data(self):
        '''
        刷新信息
        
        Returns:
            刷新信息成功返回 'refreash_data_finish!' 否則返回相應刷新失敗的信息提示
        '''

        if not self.get_account():
            return 'false_get_account'
        
        if not self.get_ticker():
            return 'false_get_ticker'
        if not self.get_depth():
            return 'false_get_depth'
        try:
            self.get_ohlc_data()
        except:
            return 'false_get_K_line_info'
        
        return 'refreash_data_finish!'


NameError: name 'PERIOD_M5' is not defined

In [3]:
class qushi_class():
    def __init__(self, mid_class, amount_N, price_N):
        '''
        設定好初始需要考慮的參數
        Args:
            mid_class: 所使用的交易所中間層
            amount_N：數量小數點限制
            price_N：價格小數點限制
            
        Attributes：
            amount_N：數量小數點限制
            price_N：價格小數點限制
            init_time：初始時間
            last_time：上一次執行操作的時間
            trade_list:交易請求的id
        '''
        self.jys = mid_class
        
        self.init_time = time.time()
        self.last_time = time.time()
        
        self.amount_N = amount_N
        self.price_N = price_N
        
        self.trade_list = []
    
    def refreash_data(self):
        '''
        用來從交易所獲取最新的價格和數量信息
        
        Attributes：
            B：商品幣數量
            money：計價幣數量
            can_buy_B：當前理論可購買商品幣數量
            Buy_price:當前市場上最近的一單掛單買價
            Sell_price：當前市場上最近的一單掛單賣價
        '''
        
        self.jys.refreash_data()
        self.B = self.jys.Amount
        self.money = self.jys.Balance
        self.Buy_price = self.jys.Buy
        self.Sell_price = self.jys.Sell
        self.can_buy_B = self.money/ self.Buy_price
    
    def make_trade_by_dict(self, trade_dicts):
        '''
        用來批量完成交易訂單
        
        Attributes：
            trade_list:已提交的交易請求的id
        '''
        for this_trade in trade_dicts:
            this_trade_id = self.jys.create_order( trade_dicts['side'], trade_dict['buy_price'] , trade_dict['amount'] ) 
            self.trade_list.append( this_trade_id )
    
    def condition_chicang(self, hands_num):
        '''
        根據持倉情況來做交易判定的條件
        Args:
            hands_num：表示交易一共幾手（我們假設當前每次交易不高於一手）
            
        Attributes：
            min_trade_B: 一手最多交易的商品幣數量
            min_trade_money: 一手最多交易的計價幣數量
        
        '''
        self.min_trade_B = (self.can_buy_B + self.B) / hands_num
        self.min_trade_money = self.min_trade_B* self.jys.Buy
        
        
    
    def condition_qushi(self, change_pct ):
        '''
        根據市場價格情況來做交易判定的條件
        Args:
            change_pct：表示價格變化多少來確定一次交易（假設根據之前一段時間均值判斷）
            
        Returns：
            min_trade_B: 一手最多交易的商品幣數量
            min_trade_money: 一手最多交易的計價幣數量
        
        '''
        mean_price = self.jys.ohlc_data[-12*24:]/(12*24)
        do_buy = self.jys.Buy > mean_price* (100.0 + change_pct )/100.0
        do_sell = self.jys.Sell < mean_price/ ((100.0 + change_pct )/100.0)
        
        if do_buy or do_sell:
            rt = 'Buy' if do_buy else 'Sell'
        
        return rt
    
    def make_trade_dicts:
        pass